In [20]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

movies_md = r'dataset/movies_tmdbMeta.csv'
movies_md_df = pd.read_csv(movies_md)
pd.set_option('display.max_columns', len(movies_md_df.columns))
movies_md_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,weighted_rating,cast_size,crew_size,director,revenue_divide_budget
0,False,Toy Story Collection,30000000.0,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,['Pixar Animation Studios'],['United States of America'],1995-10-30,373554033.0,81.0,1,Released,NaN,Toy Story,False,7.7,5415.0,"['Tom Hanks', 'Tim Allen', 'Don Rickles']","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","['jealousy', 'toy', 'boy', 'friendship', 'frie...",7.684645,13,106,John Lasseter,12.451801
1,False,NaN,65000000.0,"['Adventure', 'Fantasy', 'Family']",NaN,8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],1995-12-15,262797249.0,104.0,2,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","['board game', 'disappearance', ""based on chil...",6.876926,26,16,Joe Johnston,4.043035
2,False,Grumpy Old Men Collection,NaN,"['Romance', 'Comedy']",NaN,15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"['Warner Bros.', 'Lancaster Gate']",['United States of America'],1995-12-22,NaN,101.0,1,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret']","[{'credit_id': '52fe466a9251416c75077a89', 'de...","['fishing', 'best friend', 'duringcreditssting...",6.162328,7,4,Howard Deutch,NaN
3,False,NaN,16000000.0,"['Comedy', 'Drama', 'Romance']",NaN,31357.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,['Twentieth Century Fox Film Corporation'],['United States of America'],1995-12-22,81452156.0,127.0,1,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"['Whitney Houston', 'Angela Bassett', 'Loretta...","[{'credit_id': '52fe44779251416c91011acb', 'de...","['based on novel', 'interracial relationship',...",5.675590,10,10,Forest Whitaker,5.090760
4,False,Father of the Bride Collection,NaN,['Comedy'],NaN,11862.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"['Sandollar Productions', 'Touchstone Pictures']",['United States of America'],1995-02-10,76578911.0,106.0,1,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"['Steve Martin', 'Diane Keaton', 'Martin Short']","[{'credit_id': '52fe44959251416c75039ed7', 'de...","['baby', 'midlife crisis', 'confidence', 'agin...",5.624595,12,7,Charles Shyer,NaN


In [21]:
import ast

def list_of_str_to_str(list_of_str):
    return " ".join(list_of_str) if type(list_of_str) == list else list_of_str
    
    
def get_role_list(people, role):
    people = ast.literal_eval(people)
    crew = []
    
    for person in people:
        if person['job'] == role:
            crew.append(person['name'])
    
    return list_of_str_to_str(crew) if len(crew) else np.nan

In [11]:
# delete all rows that do not have revenue/cost recorded
print(movies_md_df.shape)
movies_md_df = movies_md_df[np.isfinite(movies_md_df['revenue_divide_budget'])]
print(movies_md_df.shape)

(46624, 32)
(5458, 32)


In [12]:
#delete non-english movies
movies_md_df = movies_md_df[movies_md_df['original_language'] == 'en']
movies_md_df = movies_md_df.drop(['original_language'], 1)

print(movies_md_df.shape)

(4851, 31)


In [13]:
'''
data modification:
- homepage -> 1/0
- add producers column

drop:
- id
- imdb_id
- poster_path
- release_date ??
- video
- weighted_rating
'''

movies_md_df = movies_md_df.drop([
    'id', 
    'imdb_id', 
    'poster_path', 
    'video', 
    'status', 
    'weighted_rating', 
    'original_title'
], 1)
print(list(movies_md_df.columns))

['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'tagline', 'title', 'vote_average', 'vote_count', 'cast', 'crew', 'keywords', 'cast_size', 'crew_size', 'director', 'revenue_divide_budget']


In [14]:
movies_md_df['homepage'] = movies_md_df['homepage'].apply(lambda x: 0 if x == np.nan else 1)
movies_md_df['producers'] = movies_md_df['crew'].apply(get_role_list, role='Producer')
movies_md_df['executive_producers'] = movies_md_df['crew'].apply(get_role_list, role='Executive Producer')

movies_md_df = movies_md_df.drop(['crew'], axis=1)

In [15]:
columns_with_list_of_str = [
    'genres', 
    'production_countries', 
    'production_companies', 
    'cast', 
    'keywords',
]

for col in columns_with_list_of_str:
    movies_md_df[col] = movies_md_df[col].apply(lambda x: list_of_str_to_str(ast.literal_eval(x)) if x != np.nan else x)
    
movies_md_df.head()

,adult,belongs_to_collection,budget,genres,homepage,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count,cast,keywords,cast_size,crew_size,director,revenue_divide_budget,producers,executive_producers
0,False,Toy Story Collection,30000000.0,Animation Comedy Family,1,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,1,NaN,Toy Story,7.7,5415.0,Tom Hanks Tim Allen Don Rickles,jealousy toy boy friendship friends rivalry bo...,13,106,John Lasseter,12.451801,Bonnie Arnold Ralph Guggenheim,Ed Catmull Steve Jobs
1,False,NaN,65000000.0,Adventure Fantasy Family,1,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,2,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst,board game disappearance based on children's b...,26,16,Joe Johnston,4.043035,Scott Kroopf William Teitler,Larry J. Franco Ted Field Robert W. Cort
3,False,NaN,16000000.0,Comedy Drama Romance,1,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,1,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine,based on novel interracial relationship single...,10,10,Forest Whitaker,5.090760,Ronald Bass Ezra Swerdlow Deborah Schindler Ca...,Terry McMillan
5,False,NaN,60000000.0,Action Crime Drama Thriller,1,"Obsessive master thief, Neil McCauley leads a ...",17.924927,Regency Enterprises Forward Pass Warner Bros.,United States of America,1995-12-15,187436818.0,170.0,2,A Los Angeles Crime Saga,Heat,7.7,1886.0,Al Pacino Robert De Niro Val Kilmer,robbery detective bank obsession chase shootin...,65,71,Michael Mann,3.123947,Art Linson Michael Mann,Pieter Jan Brugge Arnon Milchan
8,False,NaN,35000000.0,Action Adventure Thriller,1,International action superstar Jean Claude Van...,5.231580,Universal Pictures Imperial Entertainment Sign...,United States of America,1995-12-22,64350171.0,106.0,1,Terror goes into overtime.,Sudden Death,5.5,174.0,Jean-Claude Van Damme Powers Boothe Dorian Har...,terrorist hostage explosive vice president,6,9,Peter Hyams,1.838576,Moshe Diamant Anders P. Jensen Howard Baldwin,NaN


In [17]:
# drop all rows that contain non-ascii characters
import string

def is_all_ascii(chars):
    if pd.isnull(chars):
        return True
    
    printable = set(string.printable)
    for char in chars:
        if char not in printable:
            return False

    return True

text_cols = [
    'genres',
    'overview',
    'production_companies',
    'production_countries',
    'tagline',
    'title',
    'cast',
    'keywords',
    'director',
    'producers',
    'executive_producers',
    'belongs_to_collection'
]

for col in text_cols:
    movies_md_df = movies_md_df[movies_md_df[col].apply(is_all_ascii)]
    
movies_md_df.head()

,adult,belongs_to_collection,budget,genres,homepage,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count,cast,keywords,cast_size,crew_size,director,revenue_divide_budget,producers,executive_producers
0,False,Toy Story Collection,30000000.0,Animation Comedy Family,1,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,1,NaN,Toy Story,7.7,5415.0,Tom Hanks Tim Allen Don Rickles,jealousy toy boy friendship friends rivalry bo...,13,106,John Lasseter,12.451801,Bonnie Arnold Ralph Guggenheim,Ed Catmull Steve Jobs
1,False,NaN,65000000.0,Adventure Fantasy Family,1,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,United States of America,1995-12-15,262797249.0,104.0,2,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,Robin Williams Jonathan Hyde Kirsten Dunst,board game disappearance based on children's b...,26,16,Joe Johnston,4.043035,Scott Kroopf William Teitler,Larry J. Franco Ted Field Robert W. Cort
3,False,NaN,16000000.0,Comedy Drama Romance,1,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,1,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,Whitney Houston Angela Bassett Loretta Devine,based on novel interracial relationship single...,10,10,Forest Whitaker,5.090760,Ronald Bass Ezra Swerdlow Deborah Schindler Ca...,Terry McMillan
5,False,NaN,60000000.0,Action Crime Drama Thriller,1,"Obsessive master thief, Neil McCauley leads a ...",17.924927,Regency Enterprises Forward Pass Warner Bros.,United States of America,1995-12-15,187436818.0,170.0,2,A Los Angeles Crime Saga,Heat,7.7,1886.0,Al Pacino Robert De Niro Val Kilmer,robbery detective bank obsession chase shootin...,65,71,Michael Mann,3.123947,Art Linson Michael Mann,Pieter Jan Brugge Arnon Milchan
8,False,NaN,35000000.0,Action Adventure Thriller,1,International action superstar Jean Claude Van...,5.231580,Universal Pictures Imperial Entertainment Sign...,United States of America,1995-12-22,64350171.0,106.0,1,Terror goes into overtime.,Sudden Death,5.5,174.0,Jean-Claude Van Damme Powers Boothe Dorian Har...,terrorist hostage explosive vice president,6,9,Peter Hyams,1.838576,Moshe Diamant Anders P. Jensen Howard Baldwin,NaN


In [18]:
y = movies_md_df['revenue_divide_budget'].apply(lambda x: x >= 1)
y = pd.DataFrame(y.values, columns=['profitable'])
y.to_csv('dataset/Y.csv', index=True, index_label='Id')
y.head()


,profitable
0,True
1,True
2,True
3,True
4,True


In [19]:
# remove the revenue_divide_budget column once we've generate the x and y files
movies_md_df = movies_md_df.drop(['revenue_divide_budget'], 1)
movies_md_df.to_csv('dataset/X.csv', index=True, index_label='Id')